<h3>Оптимизаторы</h3>
<li> adam </li>
<li> rmsprop </li>

<h3>Функции активации</h3>
<li> relu </li>
<li> selu </li>
<li> gelu </li>
<li> elu </li>

<h3> Реккурентные слои </h3>

<li> LSTM </li>
<li> GRU </li>

<h3> Дропауты</h3>
Следи за переобучением/недообучением. Меняй значения от 0.1 до 0.5

<h3> Размерность эмбеддинга </h3>
Попробуй 64, 128, 256 (embed_dim)

<h3> Линейные слои </h3>
Добавляй / убирай Dense слои, только не трогай последний!
После Dense можно попробовать поставить layers.LayerNormalization()

<h3> Одномерные сверточные слои (Conv1D) </h3>
Меняй количество units и размер kernel size


In [1]:
from tensorflow import keras
from keras import datasets, layers, models, preprocessing
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [3]:

import tensorflow as tf


import numpy as np
from nltk import word_tokenize
from keras.utils import pad_sequences
import nltk
nltk.download('punkt')
imdb = datasets.imdb
num_words = 20000
max_len = 200
n_words = 10000
dim_embedding = 128
EPOCHS = 40 #если ты видишь, что обучаться может дальше (не вышли на плато или переобучение), то увеличь количество эпох
BATCH_SIZE = 500 #вылетает по памяти? Пробуй уменьшить batch_size
def load_data():
        # Load data.
        (X_train, y_train), (X_test, y_test) = datasets.imdb.load_data(num_words=n_words)
        # Pad sequences with max_len.
        X_train = pad_sequences(X_train, maxlen=max_len)
        X_test = pad_sequences(X_test, maxlen=max_len)
        y_train = y_train.astype(np.float64)
        y_test = y_test.astype(np.float64)
        return (X_train, y_train), (X_test, y_test)
    
def build_model1():
    model = models.Sequential()
    model.add(layers.Embedding(n_words, 
    dim_embedding, input_length=max_len))
    model.add(layers.Dropout(0.3))
    model.add(layers.LSTM(128))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model   

def build_model2():
    model = models.Sequential()
    model.add(layers.Embedding(n_words, 
    dim_embedding, input_length=max_len))
    model.add(layers.Dropout(0.3))
    model.add(layers.Bidirectional(layers.LSTM(128))) #сюда можно добавить kernel_regularizer='l2',recurrent_regularizer='l2',dropout=0.1, recurrent_dropout=0.1. В Bidirectional - merge='sum'
    #model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model  

def build_model3():
    model = models.Sequential()
    model.add(layers.Embedding(n_words, 
    dim_embedding, input_length=max_len))
    model.add(layers.Dropout(0.3))
    model.add(layers.LSTM(128,return_sequences=True)) #return sequences не уменьшает размерность, что надо для глобал макс пулинга
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model  

def build_model4():
    model = models.Sequential()
    model.add(layers.Embedding(n_words, 
    dim_embedding, input_length=max_len))
    #model.add(layers.Dropout(0.3))
    model.add(layers.LSTM(128,activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model  

def build_model5():
    model = models.Sequential()
    model.add(layers.Embedding(n_words, 
    dim_embedding, input_length=max_len))
    #model.add(layers.Dropout(0.3))
    model.add(layers.Bidirectional(layers.GRU(64,activation='relu')))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model  

def build_model6():
    model = models.Sequential()
    model.add(layers.Embedding(n_words, 
    dim_embedding, input_length=max_len))
    #model.add(layers.Dropout(0.3))
    model.add(layers.GRU(128,activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model  


def build_model7():
    model = models.Sequential()
    model.add(layers.Embedding(n_words, 
    dim_embedding, input_length=max_len))
    model.add(TransformerEncoder(dim_embedding, 128, 2))
    #model.add(layers.Dropout(0.3))
    model.add(layers.GRU(128,activation='relu'))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model  

def build_model8():
    model = models.Sequential()
    model.add(layers.Embedding(n_words, 
    dim_embedding, input_length=max_len))
    model.add(TransformerEncoder(dim_embedding, 128, 2))
    #model.add(layers.Dropout(0.3))
    model.add(layers.Conv1D(128,kernel_size=2,activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model  

def build_model8():
    model = models.Sequential()
    model.add(layers.Embedding(n_words, 
    dim_embedding, input_length=max_len))
    model.add(TransformerEncoder(dim_embedding, 128, 2))
    #model.add(layers.Dropout(0.3))
    model.add(layers.Conv1D(128,kernel_size=2,activation='relu'))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model  

(X_train, y_train), (X_test, y_test) = load_data()
model = build_model5()
model.summary()
model.compile(optimizer = "rmsprop", loss = "binary_crossentropy",
 metrics = ["accuracy", keras.metrics.F1Score(), keras.metrics.Precision(), keras.metrics.Recall()]
)
score = model.fit(X_train, y_train,
 epochs = EPOCHS,
 batch_size = BATCH_SIZE,
 validation_data = (X_test, y_test)
)
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

model.save("imdb9.h5")

val = model.predict(X_train)
print(f'результат: {np.argmax(val[0])}')
word2index = imdb.get_word_index()
test=[]
for word in word_tokenize( "i love this movie"):
     test.append(word2index[word])

test=pad_sequences([test],maxlen=200)
print(model.predict(test))

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ekaterinaaleksandrovna/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 200, 128)          1280000   
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               74496     
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1354625 (5.17 MB)
Trainable params: 1354625 (5.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/40
50/50 [==============================] - 31s 600ms/step - loss: 0.6918 

/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


782/782 [==============================] - 20s 25ms/step
результат: 0
1/1 [==============================] - 0s 17ms/step
[[0.46335655]]


In [5]:
model = build_model1()
model.summary()
model.compile(optimizer = "rmsprop", loss = "binary_crossentropy",
 metrics = ["accuracy", keras.metrics.F1Score(), keras.metrics.Precision(), keras.metrics.Recall()]
)
score = model.fit(X_train, y_train,
 epochs = EPOCHS,
 batch_size = BATCH_SIZE,
 validation_data = (X_test, y_test)
)
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

model.save("imdb1.h5")

val = model.predict(X_train)
print(f'результат: {np.argmax(val[0])}')
word2index = imdb.get_word_index()
test=[]
for word in word_tokenize( "i love this movie"):
     test.append(word2index[word])

test=pad_sequences([test],maxlen=200)
print(model.predict(test))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 200, 128)          1280000   
                                                                 
 dropout_4 (Dropout)         (None, 200, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1428225 (5.45 MB)
Trainable params: 1428

/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


782/782 [==============================] - 35s 44ms/step
результат: 0
1/1 [==============================] - 0s 28ms/step
[[0.00269105]]


In [6]:
model = build_model2()
model.summary()
model.compile(optimizer = "rmsprop", loss = "binary_crossentropy",
 metrics = ["accuracy", keras.metrics.F1Score(), keras.metrics.Precision(), keras.metrics.Recall()]
)
score = model.fit(X_train, y_train,
 epochs = EPOCHS,
 batch_size = BATCH_SIZE,
 validation_data = (X_test, y_test)
)
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

model.save("imdb2.h5")

val = model.predict(X_train)
print(f'результат: {np.argmax(val[0])}')
word2index = imdb.get_word_index()
test=[]
for word in word_tokenize( "i love this movie"):
     test.append(word2index[word])

test=pad_sequences([test],maxlen=200)
print(model.predict(test))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 200, 128)          1280000   
                                                                 
 dropout_6 (Dropout)         (None, 200, 128)          0         
                                                                 
 bidirectional_2 (Bidirecti  (None, 256)               263168    
 onal)                                                           
                                                                 
 dense_6 (Dense)             (None, 128)               32896     
                                                                 
 dropout_7 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 1)                 129       
                                                      

/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


782/782 [==============================] - 47s 59ms/step
результат: 0
1/1 [==============================] - 0s 44ms/step
[[0.01985721]]


In [7]:
model = build_model3()
model.summary()
model.compile(optimizer = "rmsprop", loss = "binary_crossentropy",
 metrics = ["accuracy", keras.metrics.F1Score(), keras.metrics.Precision(), keras.metrics.Recall()]
)
score = model.fit(X_train, y_train,
 epochs = EPOCHS,
 batch_size = BATCH_SIZE,
 validation_data = (X_test, y_test)
)
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

model.save("imdb3.h5")

val = model.predict(X_train)
print(f'результат: {np.argmax(val[0])}')
word2index = imdb.get_word_index()
test=[]
for word in word_tokenize( "i love this movie"):
     test.append(word2index[word])

test=pad_sequences([test],maxlen=200)
print(model.predict(test))

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 200, 128)          1280000   
                                                                 
 dropout_8 (Dropout)         (None, 200, 128)          0         
                                                                 
 lstm_3 (LSTM)               (None, 200, 128)          131584    
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense_8 (Dense)             (None, 128)               16512     
                                                                 
 dropout_9 (Dropout)         (None, 128)               0         
                                                      

In [9]:
model = build_model4()
model.summary()
model.compile(optimizer = "rmsprop", loss = "binary_crossentropy",
 metrics = ["accuracy", keras.metrics.F1Score(), keras.metrics.Precision(), keras.metrics.Recall()]
)
score = model.fit(X_train, y_train,
 epochs = EPOCHS,
 batch_size = BATCH_SIZE,
 validation_data = (X_test, y_test)
)
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

model.save("imdb4.h5")

val = model.predict(X_train)
print(f'результат: {np.argmax(val[0])}')
word2index = imdb.get_word_index()
test=[]
for word in word_tokenize( "i love this movie"):
     test.append(word2index[word])

test=pad_sequences([test],maxlen=200)
print(model.predict(test))

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 200, 128)          1280000   
                                                                 
 lstm_5 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_11 (Dropout)        (None, 128)               0         
                                                                 
 dense_11 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1411713 (5.39 MB)
Trainable params: 1411713 (5.39 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/40
50/50 [==============================] - 47s 925ms/step - loss: 0.6920 - accuracy: 0.5342 - f1_score: 0.6667 - precision_7: 0.5519 - reca

In [10]:
model = build_model6()
model.summary()
model.compile(optimizer = "rmsprop", loss = "binary_crossentropy",
 metrics = ["accuracy", keras.metrics.F1Score(), keras.metrics.Precision(), keras.metrics.Recall()]
)
score = model.fit(X_train, y_train,
 epochs = EPOCHS,
 batch_size = BATCH_SIZE,
 validation_data = (X_test, y_test)
)
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

model.save("imdb6.h5")

val = model.predict(X_train)
print(f'результат: {np.argmax(val[0])}')
word2index = imdb.get_word_index()
test=[]
for word in word_tokenize( "i love this movie"):
     test.append(word2index[word])

test=pad_sequences([test],maxlen=200)
print(model.predict(test))

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 200, 128)          1280000   
                                                                 
 gru_2 (GRU)                 (None, 128)               99072     
                                                                 
 dropout_12 (Dropout)        (None, 128)               0         
                                                                 
 dense_12 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1379201 (5.26 MB)
Trainable params: 1379201 (5.26 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/40
50/50 [==============================] - 37s 724ms/step - loss: 0.6905 - accuracy: 0.5365 - f1_score: 0.6667 - precision_8: 0.5542 - reca

In [11]:
model = build_model7()
model.summary()
model.compile(optimizer = "rmsprop", loss = "binary_crossentropy",
 metrics = ["accuracy", keras.metrics.F1Score(), keras.metrics.Precision(), keras.metrics.Recall()]
)
score = model.fit(X_train, y_train,
 epochs = EPOCHS,
 batch_size = BATCH_SIZE,
 validation_data = (X_test, y_test)
)
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

model.save("imdb6.h5")

val = model.predict(X_train)
print(f'результат: {np.argmax(val[0])}')
word2index = imdb.get_word_index()
test=[]
for word in word_tokenize( "i love this movie"):
     test.append(word2index[word])

test=pad_sequences([test],maxlen=200)
print(model.predict(test))

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 200, 128)          1280000   
                                                                 
 transformer_encoder (Trans  (None, 200, 128)          165504    
 formerEncoder)                                                  
                                                                 
 gru_3 (GRU)                 (None, 128)               99072     
                                                                 
 dropout_13 (Dropout)        (None, 128)               0         
                                                                 
 dense_15 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1544705 (5.89 MB)
Trainable params: 1544705 (5.89 MB)
Non-trainable params: 0 (0.00 Byte)
________________

/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


782/782 [==============================] - 54s 69ms/step
результат: 0
1/1 [==============================] - 0s 17ms/step
[[0.20231257]]


In [12]:
model = build_model8()
model.summary()
model.compile(optimizer = "rmsprop", loss = "binary_crossentropy",
 metrics = ["accuracy", keras.metrics.F1Score(), keras.metrics.Precision(), keras.metrics.Recall()]
)
score = model.fit(X_train, y_train,
 epochs = EPOCHS,
 batch_size = BATCH_SIZE,
 validation_data = (X_test, y_test)
)
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

model.save("imdb6.h5")

val = model.predict(X_train)
print(f'результат: {np.argmax(val[0])}')
word2index = imdb.get_word_index()
test=[]
for word in word_tokenize( "i love this movie"):
     test.append(word2index[word])

test=pad_sequences([test],maxlen=200)
print(model.predict(test))

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 200, 128)          1280000   
                                                                 
 transformer_encoder_1 (Tra  (None, 200, 128)          165504    
 nsformerEncoder)                                                
                                                                 
 conv1d (Conv1D)             (None, 199, 128)          32896     
                                                                 
 flatten (Flatten)           (None, 25472)             0         
                                                                 
 dropout_14 (Dropout)        (None, 25472)             0         
                                                                 
 dense_18 (Dense)            (None, 1)                 25473     
                                                     

/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


782/782 [==============================] - 31s 39ms/step
результат: 0
1/1 [==============================] - 0s 14ms/step
[[0.4550272]]


In [13]:
model = build_model1()
model.summary()
model.compile(optimizer = "adam", loss = "binary_crossentropy",
 metrics = ["accuracy", keras.metrics.F1Score(), keras.metrics.Precision(), keras.metrics.Recall()]
)
score = model.fit(X_train, y_train,
 epochs = EPOCHS,
 batch_size = BATCH_SIZE,
 validation_data = (X_test, y_test)
)
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

model.save("imdb1.h5")

val = model.predict(X_train)
print(f'результат: {np.argmax(val[0])}')
word2index = imdb.get_word_index()
test=[]
for word in word_tokenize( "i love this movie"):
     test.append(word2index[word])

test=pad_sequences([test],maxlen=200)
print(model.predict(test))

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 200, 128)          1280000   
                                                                 
 dropout_15 (Dropout)        (None, 200, 128)          0         
                                                                 
 lstm_6 (LSTM)               (None, 128)               131584    
                                                                 
 dense_19 (Dense)            (None, 128)               16512     
                                                                 
 dropout_16 (Dropout)        (None, 128)               0         
                                                                 
 dense_20 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1428225 (5.45 MB)
Trainable params: 142

In [14]:
model = build_model2()
model.summary()
model.compile(optimizer = "adam", loss = "binary_crossentropy",
 metrics = ["accuracy", keras.metrics.F1Score(), keras.metrics.Precision(), keras.metrics.Recall()]
)
score = model.fit(X_train, y_train,
 epochs = EPOCHS,
 batch_size = BATCH_SIZE,
 validation_data = (X_test, y_test)
)
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

model.save("imdb2.h5")

val = model.predict(X_train)
print(f'результат: {np.argmax(val[0])}')
word2index = imdb.get_word_index()
test=[]
for word in word_tokenize( "i love this movie"):
     test.append(word2index[word])

test=pad_sequences([test],maxlen=200)
print(model.predict(test))

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 200, 128)          1280000   
                                                                 
 dropout_17 (Dropout)        (None, 200, 128)          0         
                                                                 
 bidirectional_3 (Bidirecti  (None, 256)               263168    
 onal)                                                           
                                                                 
 dense_21 (Dense)            (None, 128)               32896     
                                                                 
 dropout_18 (Dropout)        (None, 128)               0         
                                                                 
 dense_22 (Dense)            (None, 1)                 129       
                                                     

/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


782/782 [==============================] - 44s 56ms/step
результат: 0
1/1 [==============================] - 0s 28ms/step
[[0.01304642]]


In [15]:
model = build_model3()
model.summary()
model.compile(optimizer = "adam", loss = "binary_crossentropy",
 metrics = ["accuracy", keras.metrics.F1Score(), keras.metrics.Precision(), keras.metrics.Recall()]
)
score = model.fit(X_train, y_train,
 epochs = EPOCHS,
 batch_size = BATCH_SIZE,
 validation_data = (X_test, y_test)
)
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

model.save("imdb2.h5")

val = model.predict(X_train)
print(f'результат: {np.argmax(val[0])}')
word2index = imdb.get_word_index()
test=[]
for word in word_tokenize( "i love this movie"):
     test.append(word2index[word])

test=pad_sequences([test],maxlen=200)
print(model.predict(test))

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 200, 128)          1280000   
                                                                 
 dropout_19 (Dropout)        (None, 200, 128)          0         
                                                                 
 lstm_8 (LSTM)               (None, 200, 128)          131584    
                                                                 
 global_max_pooling1d_1 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_23 (Dense)            (None, 128)               16512     
                                                                 
 dropout_20 (Dropout)        (None, 128)               0         
                                                     

In [16]:
model = build_model4()
model.summary()
model.compile(optimizer = "adam", loss = "binary_crossentropy",
 metrics = ["accuracy", keras.metrics.F1Score(), keras.metrics.Precision(), keras.metrics.Recall()]
)
score = model.fit(X_train, y_train,
 epochs = EPOCHS,
 batch_size = BATCH_SIZE,
 validation_data = (X_test, y_test)
)
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

model.save("imdb2.h5")

val = model.predict(X_train)
print(f'результат: {np.argmax(val[0])}')
word2index = imdb.get_word_index()
test=[]
for word in word_tokenize( "i love this movie"):
     test.append(word2index[word])

test=pad_sequences([test],maxlen=200)
print(model.predict(test))

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_14 (Embedding)    (None, 200, 128)          1280000   
                                                                 
 lstm_9 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_21 (Dropout)        (None, 128)               0         
                                                                 
 dense_25 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1411713 (5.39 MB)
Trainable params: 1411713 (5.39 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/40
50/50 [==============================] - 43s 852ms/step - loss: nan - accuracy: 0.5610 - f1_score: 0.5274 - precision_14: 0.6357 - recal

In [17]:
model = build_model5()
model.summary()
model.compile(optimizer = "adam", loss = "binary_crossentropy",
 metrics = ["accuracy", keras.metrics.F1Score(), keras.metrics.Precision(), keras.metrics.Recall()]
)
score = model.fit(X_train, y_train,
 epochs = EPOCHS,
 batch_size = BATCH_SIZE,
 validation_data = (X_test, y_test)
)
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

model.save("imdb2.h5")

val = model.predict(X_train)
print(f'результат: {np.argmax(val[0])}')
word2index = imdb.get_word_index()
test=[]
for word in word_tokenize( "i love this movie"):
     test.append(word2index[word])

test=pad_sequences([test],maxlen=200)
print(model.predict(test))

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 200, 128)          1280000   
                                                                 
 bidirectional_4 (Bidirecti  (None, 128)               74496     
 onal)                                                           
                                                                 
 dropout_22 (Dropout)        (None, 128)               0         
                                                                 
 dense_26 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1354625 (5.17 MB)
Trainable params: 1354625 (5.17 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/40
50/50 [==============================] - 39s 751ms/step - loss: 0.6690

In [18]:
model = build_model6()
model.summary()
model.compile(optimizer = "adam", loss = "binary_crossentropy",
 metrics = ["accuracy", keras.metrics.F1Score(), keras.metrics.Precision(), keras.metrics.Recall()]
)
score = model.fit(X_train, y_train,
 epochs = EPOCHS,
 batch_size = BATCH_SIZE,
 validation_data = (X_test, y_test)
)
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

model.save("imdb2.h5")

val = model.predict(X_train)
print(f'результат: {np.argmax(val[0])}')
word2index = imdb.get_word_index()
test=[]
for word in word_tokenize( "i love this movie"):
     test.append(word2index[word])

test=pad_sequences([test],maxlen=200)
print(model.predict(test))

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 200, 128)          1280000   
                                                                 
 gru_5 (GRU)                 (None, 128)               99072     
                                                                 
 dropout_23 (Dropout)        (None, 128)               0         
                                                                 
 dense_27 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1379201 (5.26 MB)
Trainable params: 1379201 (5.26 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/40
50/50 [==============================] - 36s 689ms/step - loss: 3963381325234176.0000 - accuracy: 0.5882 - f1_score: 0.6674 - precision_

In [19]:
model = build_model7()
model.summary()
model.compile(optimizer = "adam", loss = "binary_crossentropy",
 metrics = ["accuracy", keras.metrics.F1Score(), keras.metrics.Precision(), keras.metrics.Recall()]
)
score = model.fit(X_train, y_train,
 epochs = EPOCHS,
 batch_size = BATCH_SIZE,
 validation_data = (X_test, y_test)
)
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

model.save("imdb2.h5")

val = model.predict(X_train)
print(f'результат: {np.argmax(val[0])}')
word2index = imdb.get_word_index()
test=[]
for word in word_tokenize( "i love this movie"):
     test.append(word2index[word])

test=pad_sequences([test],maxlen=200)
print(model.predict(test))

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, 200, 128)          1280000   
                                                                 
 transformer_encoder_2 (Tra  (None, 200, 128)          165504    
 nsformerEncoder)                                                
                                                                 
 gru_6 (GRU)                 (None, 128)               99072     
                                                                 
 dropout_24 (Dropout)        (None, 128)               0         
                                                                 
 dense_30 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1544705 (5.89 MB)
Trainable params: 1544705 (5.89 MB)
Non-trainable params: 0 (0.00 Byte)
_______________

In [20]:
model = build_model8()
model.summary()
model.compile(optimizer = "adam", loss = "binary_crossentropy",
 metrics = ["accuracy", keras.metrics.F1Score(), keras.metrics.Precision(), keras.metrics.Recall()]
)
score = model.fit(X_train, y_train,
 epochs = EPOCHS,
 batch_size = BATCH_SIZE,
 validation_data = (X_test, y_test)
)
score = model.evaluate(X_test, y_test, batch_size=BATCH_SIZE)
print("\nTest score:", score[0])
print('Test accuracy:', score[1])

model.save("imdb2.h5")

val = model.predict(X_train)
print(f'результат: {np.argmax(val[0])}')
word2index = imdb.get_word_index()
test=[]
for word in word_tokenize( "i love this movie"):
     test.append(word2index[word])

test=pad_sequences([test],maxlen=200)
print(model.predict(test))

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_18 (Embedding)    (None, 200, 128)          1280000   
                                                                 
 transformer_encoder_3 (Tra  (None, 200, 128)          165504    
 nsformerEncoder)                                                
                                                                 
 conv1d_1 (Conv1D)           (None, 199, 128)          32896     
                                                                 
 flatten_1 (Flatten)         (None, 25472)             0         
                                                                 
 dropout_25 (Dropout)        (None, 25472)             0         
                                                                 
 dense_33 (Dense)            (None, 1)                 25473     
                                                     

SWITCH

In [22]:
from tensorflow import keras
from keras import ops
from keras import layers

vocab_size = 10000  # Only consider the top 20k words
num_tokens_per_example = 200  # Only consider the first 200 words of each movie review
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.utils.pad_sequences(x_train, maxlen=num_tokens_per_example)
x_val = keras.utils.pad_sequences(x_val, maxlen=num_tokens_per_example)

embed_dim = 64  # Embedding size for each token.
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feedforward network.
num_experts = 10  # Number of experts used in the Switch Transformer.
batch_size = 50  # Batch size.
learning_rate = 0.001  # Learning rate.
dropout_rate = 0.3  # Dropout rate.
num_epochs = 10 # Number of epochs.
num_tokens_per_batch = (
    batch_size * num_tokens_per_example
)  # Total number of tokens per batch.
print(f"Number of tokens per batch: {num_tokens_per_batch}")


ImportError: cannot import name 'ops' from 'keras' (/opt/homebrew/lib/python3.11/site-packages/keras/__init__.py)

In [31]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = ops.shape(x)[-1]
        positions = ops.arange(start=0, stop=maxlen, step=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [33]:
def create_feedforward_network(ff_dim, embed_dim, name=None):
    return keras.Sequential(
        [layers.Dense(ff_dim, activation="gelu"), layers.Dense(embed_dim)], name=name
    )

In [34]:
def load_balanced_loss(router_probs, expert_mask):
    # router_probs [tokens_per_batch, num_experts] is the probability assigned for
    # each expert per token. expert_mask [tokens_per_batch, num_experts] contains
    # the expert with the highest router probability in one−hot format.

    num_experts = ops.shape(expert_mask)[-1]
    # Get the fraction of tokens routed to each expert.
    # density is a vector of length num experts that sums to 1.
    density = ops.mean(expert_mask, axis=0)
    # Get fraction of probability mass assigned to each expert from the router
    # across all tokens. density_proxy is a vector of length num experts that sums to 1.
    density_proxy = ops.mean(router_probs, axis=0)
    # Want both vectors to have uniform allocation (1/num experts) across all
    # num_expert elements. The two vectors will be pushed towards uniform allocation
    # when the dot product is minimized.
    loss = ops.mean(density_proxy * density) * ops.cast((num_experts**2), "float32")
    return loss

In [35]:
class Router(layers.Layer):
    def __init__(self, num_experts, expert_capacity):
        self.num_experts = num_experts
        self.route = layers.Dense(units=num_experts)
        self.expert_capacity = expert_capacity
        super().__init__()

    def call(self, inputs, training=False):
        # inputs shape: [tokens_per_batch, embed_dim]
        # router_logits shape: [tokens_per_batch, num_experts]
        router_logits = self.route(inputs)

        if training:
            # Add noise for exploration across experts.
            router_logits += keras.random.uniform(
                shape=router_logits.shape, minval=0.9, maxval=1.1
            )
        # Probabilities for each token of what expert it should be sent to.
        router_probs = keras.activations.softmax(router_logits, axis=-1)
        # Get the top−1 expert for each token. expert_gate is the top−1 probability
        # from the router for each token. expert_index is what expert each token
        # is going to be routed to.
        expert_gate, expert_index = ops.top_k(router_probs, k=1)
        # expert_mask shape: [tokens_per_batch, num_experts]
        expert_mask = ops.one_hot(expert_index, self.num_experts)
        # Compute load balancing loss.
        aux_loss = load_balanced_loss(router_probs, expert_mask)
        self.add_loss(aux_loss)
        # Experts have a fixed capacity, ensure we do not exceed it. Construct
        # the batch indices, to each expert, with position in expert make sure that
        # not more that expert capacity examples can be routed to each expert.
        position_in_expert = ops.cast(
            ops.cumsum(expert_mask, axis=0) * expert_mask, "int32"
        )
        # Keep only tokens that fit within expert capacity.
        expert_mask *= ops.cast(
            ops.less(ops.cast(position_in_expert, "int32"), self.expert_capacity),
            "float32",
        )
        expert_mask_flat = ops.sum(expert_mask, axis=-1)
        # Mask out the experts that have overflowed the expert capacity.
        expert_gate *= expert_mask_flat
        # Combine expert outputs and scaling with router probability.
        # combine_tensor shape: [tokens_per_batch, num_experts, expert_capacity]
        combined_tensor = ops.expand_dims(
            expert_gate
            * expert_mask_flat
            * ops.squeeze(ops.one_hot(expert_index, self.num_experts), 1),
            -1,
        ) * ops.squeeze(ops.one_hot(position_in_expert, self.expert_capacity), 1)
        # Create binary dispatch_tensor [tokens_per_batch, num_experts, expert_capacity]
        # that is 1 if the token gets routed to the corresponding expert.
        dispatch_tensor = ops.cast(combined_tensor, "float32")

        return dispatch_tensor, combined_tensor

In [36]:
class Switch(layers.Layer):
    def __init__(
        self, num_experts, embed_dim, ff_dim, num_tokens_per_batch, capacity_factor=1
    ):
        self.num_experts = num_experts
        self.embed_dim = embed_dim
        self.experts = [
            create_feedforward_network(ff_dim, embed_dim) for _ in range(num_experts)
        ]

        self.expert_capacity = num_tokens_per_batch // self.num_experts
        self.router = Router(self.num_experts, self.expert_capacity)
        super().__init__()

    def call(self, inputs):
        batch_size = ops.shape(inputs)[0]
        num_tokens_per_example = ops.shape(inputs)[1]

        # inputs shape: [num_tokens_per_batch, embed_dim]
        inputs = ops.reshape(inputs, [num_tokens_per_batch, self.embed_dim])
        # dispatch_tensor shape: [expert_capacity, num_experts, tokens_per_batch]
        # combine_tensor shape: [tokens_per_batch, num_experts, expert_capacity]
        dispatch_tensor, combine_tensor = self.router(inputs)
        # expert_inputs shape: [num_experts, expert_capacity, embed_dim]
        expert_inputs = ops.einsum("ab,acd->cdb", inputs, dispatch_tensor)
        expert_inputs = ops.reshape(
            expert_inputs, [self.num_experts, self.expert_capacity, self.embed_dim]
        )
        # Dispatch to experts
        expert_input_list = ops.unstack(expert_inputs, axis=0)
        expert_output_list = [
            self.experts[idx](expert_input)
            for idx, expert_input in enumerate(expert_input_list)
        ]
        # expert_outputs shape: [expert_capacity, num_experts, embed_dim]
        expert_outputs = ops.stack(expert_output_list, axis=1)
        # expert_outputs_combined shape: [tokens_per_batch, embed_dim]
        expert_outputs_combined = ops.einsum(
            "abc,xba->xc", expert_outputs, combine_tensor
        )
        # output shape: [batch_size, num_tokens_per_example, embed_dim]
        outputs = ops.reshape(
            expert_outputs_combined,
            [batch_size, num_tokens_per_example, self.embed_dim],
        )
        return outputs

In [37]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ffn, dropout_rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        # The ffn can be either a standard feedforward network or a switch
        # layer with a Mixture of Experts.
        self.ffn = ffn
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)

    def call(self, inputs, training=False):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [38]:
def create_classifier():
    switch = Switch(num_experts, embed_dim, ff_dim, num_tokens_per_batch)
    transformer_block = TransformerBlock(embed_dim // num_heads, num_heads, switch)

    inputs = layers.Input(shape=(num_tokens_per_example,))
    embedding_layer = TokenAndPositionEmbedding(
        num_tokens_per_example, vocab_size, embed_dim
    )
    x = embedding_layer(inputs)
    x = transformer_block(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(ff_dim, activation="relu")(x) 
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(2, activation="softmax")(x)

    classifier = keras.Model(inputs=inputs, outputs=outputs)
    return classifier

In [39]:
def run_experiment(classifier):
    classifier.compile(
        optimizer=keras.optimizers.Adam(learning_rate), #поменять на RMSProp
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    history = classifier.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=num_epochs,
        validation_data=(x_val, y_val),
    )
    return history


classifier = create_classifier()
run_experiment(classifier)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 325s 641ms/step - accuracy: 0.6665 - loss: 1.5749 - val_accuracy: 0.8788 - val_loss: 1.2904
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 317s 634ms/step - accuracy: 0.9211 - loss: 1.2108 - val_accuracy: 0.8704 - val_loss: 1.3028
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 315s 629ms/step - accuracy: 0.9580 - loss: 1.1251 - val_accuracy: 0.8611 - val_loss: 1.3653
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 310s 621ms/step - accuracy: 0.9778 - loss: 1.0737 - val_accuracy: 0.8524 - val_loss: 1.5357
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 364s 728ms/step - accuracy: 0.9868 - loss: 1.0430 - val_accuracy: 0.8382 - val_loss: 1.6533
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 319s 638ms/step - accuracy: 0.9903 - loss: 1.0326 - val_accuracy: 0.8446 - val_loss: 1.8236
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 325s 649ms/step - accuracy: 0.9927 - loss: 1.0241 - val_accuracy: 0.8428 - val_loss: 1.8575
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 314s 628ms/step - accuracy: 0.9951 -

TRANSFORMER

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = ops.shape(x)[-1]
        positions = ops.arange(start=0, stop=maxlen, step=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
vocab_size = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.utils.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.utils.pad_sequences(x_val, maxlen=maxlen)

In [ ]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val)
)